## Working with compute contexts in Azure ML

### Introduction to Environment

### Creating Environments

##### Créer un env à partir d'un fichier de spécification

1- créer un fichier **conda.yml** avec les paramètres de configuration:

In [ ]:
name: py_env
dependencies:
  - numpy
  - pandas
  - scikit-learn
  - pip:
    - azureml-defaults

2- Créer un env Azure ML à partir du fichiers de spec enregistrées:

In [ ]:
from azureml.core import Environment

env = Environment.from_conda_specification(name='training_environment',
                                           file_path='./conda.yml')

##### Création d'un environement à partir d'un Conda existant 

In [ ]:
from azureml.core import Environment

env = Environment.from_existing_conda_environment(name='training_environment',
                                                  conda_environment_name='py_env') #correspond au nom spécifié du 
                                                                                   #fichier conda.yml

##### Création d'un evnironement en spécifiant les packages 

Objet = **CondaDependencies** => offre la possibilité de spécifier les packages conda et pip. 

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment('training_environment')
deps = CondaDependencies.create(conda_packages=['scikit-learn','pandas','numpy'],
                                pip_packages=['azureml-defaults'])


env.python.conda_dependencies = deps

In [ ]:
#exercice exemple:

from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
diabetes_env.docker.enabled = True # Use a docker container

# Create a set of package dependencies (conda or pip as required)
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn'],
                                          pip_packages=['azureml-defaults', 'azureml-dataprep[pandas]'])

# Add the dependencies to the environment
diabetes_env.python.conda_dependencies = diabetes_packages

print(diabetes_env.name, 'defined.')

### Enregistrement et réutilsation des environements

- où ? : dans un ET
- utilisable pour de futurs expériences ayant même dépendance python

##### Enregistrement

1- Enregsitrement 

In [ ]:
env.register(workspace=ws)

2- Affichage des environnements enregistrés dans l'ET 

In [ ]:
from azureml.core import Environment

env_names = Environment.list(workspace=ws)


for env_name in env_names:
    print('Name:',env_name)

##### Recuperation et utllisation d'un environement

In [ ]:
from azureml.core import Environment, Estimator

training_env = Environment.get(workspace=ws, name='training_environment')
estimator = Estimator(source_directory='experiment_folder'
                      entry_script='training_script.py',
                      compute_target='local',
                      environment_definition=training_env)

Si spéc d'un environement, Azure ML recherchera un environement existant qui correspond à la définition 
Si spéc non trouvée  => nouvel env sera créer en fonctio de la spé d'env enregistré. 

### Introduction to compute target

**defintion compute target**: ordinateurs physiques ou virtuels sur lesquels des expériences sont exécutées.Cela offre la possibilité d'affecter des exécutions d'expériences à des cibles de calcul spécifiques

L'un des principaux avantages du cloud computing est la capacité de gérer les coûts en ne payant que pour ce que vous utilisez. Dans Azure Machine Learning, vous pouvez tirer parti de ce principe en définissant des cibles de calcul qui:

- Démarrez à la demande et arrêtez automatiquement lorsqu'il n'est plus nécessaire.
- Évoluez automatiquement en fonction des besoins de traitement de la charge de travail

##### Type de calcul

- <font color="red">1) Calcul local</font>

Cela exécute l'expérience sur la même cible de calcul que le code utilisé pour lancer l'expérience, qui peut être votre poste de travail physique ou une machine virtuelle telle qu'une instance de calcul Azure Machine Learning sur laquelle vous exécutez un bloc-notes.
Le calcul local est généralement un excellent choix lors du développement et des tests avec des volumes de données faibles à modérés.

- <font color="red">2) Training Clusters</font>

Pour les charges de travail de formation avec des exigences d'évolutivité élevées, vous pouvez utiliser des clusters de formation Azure Machine Learning; qui sont des clusters multi-nœuds de machines virtuelles qui évoluent automatiquement vers le haut ou vers le bas pour répondre à la demande.

avantage :
- rentable pour executer exp avec de gros volumes de données
- permet un traitement parallèle pour répartir la charge de travail et réduire le temps d'exécution

- <font color="red">3) Clusters d'inférence</font>

Pour déployer des modèles formés en tant que services de production, vous pouvez utiliser des clusters d'inférence Azure Machine Learning, qui utilisent des technologies de conteneurisation pour permettre une initialisation rapide du calcul pour l'inférence à la demande.

- <font color="red">4) Attached Compute</font>

Si vous utilisez déjà un environnement de calcul basé sur Azure pour la science des données, tel qu'une machine virtuelle ou un cluster Azure Databricks, vous pouvez le joindre à votre espace de travail Azure Machine Learning et l'utiliser comme cible de calcul pour certains types de charge de travail.

### Créer des compute target 

##### Create Managed compute target

Definition:A **managed compute target** is one that is managed by Azure Machine Learning, such as an Azure Machine Learning training cluster.

In [ ]:
#1 vérification de la présence d'une compute target sinon création

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=4)
    training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

training_cluster.wait_for_completion(show_output=True)

Exemple de création de Azure ML training cluster compute target via deux classes:
- azureml.core.ComputeTarget 
- azureml.core.AmlCompute

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute

# Load the workspace from the saved config file
ws = Workspace.from_config()

# Specify a name for the compute (unique within the workspace)
compute_name = 'aml-cluster'

# Define compute configuration
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                       min_nodes=0, max_nodes=4,
                                                       vm_priority='dedicated')

# Create the compute
aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)

aml_cluster.wait_for_completion(show_output=True)

Here, creation of : 
- a cluster with up to four nodes 
- based on the STANDARD_DS12_v2 virtual machine image 
- The priority for the virtual machines (VMs) is set to **dedicated** : meaning they are reserved for use in this cluster 

alternative to dedicated is **lowpriority**: which has a lower cost but means that the VMs can be preempted if a higher-priority workload requires the compute)

##### Create unmanaged compute target

An **unmanaged compute target** is one that is defined and managed outside of the Azure Machine Learning workspace; for example, an Azure virtual machine or an Azure Databricks cluster.

méthode **ComputeTarget.attach ()** pour attacher le calcul existant en fonction de ses paramètres de configuration spécifiques à la cible

In [ ]:
#exemple pour ajouter/attacher un cluster databricks existant

from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, DatabricksCompute

# Load the workspace from the saved config file
ws = Workspace.from_config()

# Specify a name for the compute (unique within the workspace)
compute_name = 'db_cluster'

# Define configuration for existing Azure Databricks cluster
db_workspace_name = 'db_workspace'
db_resource_group = 'db_resource_group'
db_access_token = '1234-abc-5678-defg-90...'
db_config = DatabricksCompute.attach_configuration(resource_group=db_resource_group,
                                                   workspace_name=db_workspace_name,
                                                   access_token=db_access_token)

# Create the compute
databricks_compute = ComputeTarget.attach(ws, compute_name, db_config)
databricks_compute.wait_for_completion(True)

##### Checking for an existing compute target


ComputeTargetException

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_name = "aml-cluster"

# Check if the compute target exists
try:
    aml_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)

aml_cluster.wait_for_completion(show_output=True)

##### En général sur les compute target

When you run a script as an Azure Machine Learning experiment, you need to define the execution context for the experiment run. The execution context is made up of:

- The Python **environment** for the script, which must include **all Python packages used in the script**.

- The **compute target** on which the script will be run (**local workstation** from which the experiment run is initiated, or **compute target** (exemple: training cluster that is provisioned on-demand).

Défine an environment :

When you run a Python script as an experiment in Azure Machine Learning, a Conda environment is created to define the execution context for the script.

Azure Machine Learning provides a default environment that includes common packages: ex => azureml-defaults package that contains the libraries necessary for working with an experiment run,

In a typical model development lifecycle, you might:

- **Start by developing and experimenting on a small amount of data**. At this stage, we recommend your **local environment (local computer or cloud-based VM)** as your compute target.

- **Scale up to larger data, or do distributed training** using one of these **training compute targets** (Azure Databricks, Azure Data Lake Analytics, Azure Machine Learning compute cluster, Azure Batch etc...)

- Once your **model is ready**, **deploy it to a web hosting environment or IoT device with one of these deployment compute targets.** (Azure Machine Learning compute instance web service, Azure Kubernetes Service (AKS), Azure Container Instances etc...)

### Utiliser des compute targets

After you've created environments and compute targets in your workspace, you can use them to run specific workloads; such as experiments.

example, the following code configures an estimator to **use the compute target named** aml-cluster:

In [ ]:
from azureml.core import Environment, Estimator

compute_name = 'aml-cluster'

training_env = Environment.get(workspace=ws, name='training_environment')

estimator = Estimator(source_directory='experiment_folder',
                      entry_script='training_script.py',
                      environment_definition=training_env,
                      compute_target=compute_name) 

Instead of specifying the name of the compute target, you can specify a **ComputeTarget object**, like this:

In [ ]:
from azureml.core import Environment, Estimator
from azureml.core.compute import ComputeTarget

compute_name = 'aml-cluster'
**training_cluster** = ComputeTarget(workspace=ws, name=compute_name)



training_env = Environment.get(workspace=ws, name='training_environment')

estimator = Estimator(source_directory='experiment_folder',
                      entry_script='training_script.py',
                      environment_definition=training_env,
                      compute_target=**training_cluster**)